### 네이버 영화 리뷰 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')

In [3]:
print(train_df.shape, test_df.shape)
train_df.head()

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


##### 1. 데이터 전처리

- train data set

In [4]:
# 결측치가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [5]:
# 결측치 데이터 삭제
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [6]:
# 중복 데이터 확인
train_df.document.nunique()

146182

In [7]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [8]:
# 데이터 분포
train_df.label.value_counts()

label
0    73342
1    72840
Name: count, dtype: int64

- 테스트 데이터 셋

In [9]:
test_df.dropna(how='any', inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [10]:
test_df.label.value_counts()

label
1    24711
0    24446
Name: count, dtype: int64

##### 2. 텍스트 전처리

- 트레인 데이터 셋

In [11]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# 영어 데이터 --> '' 만 남게됨 --> np.nan으로 대체한 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [13]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- 테스트 데이터 셋

In [14]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

In [15]:
train_df.to_csv('../data/네이버영화리뷰_train_전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('../data/네이버영화리뷰_test_전처리완료.tsv', sep='\t', index=False)

##### 3. 한글 처리

In [16]:
from konlpy.tag import Okt
okt = Okt()

In [17]:
text = '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [18]:
# 동사 원형 stem=True
okt.morphs(text, stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [20]:
with open('../data/한글불용어 copy.txt', encoding='utf-8') as st:
    lines = st.readlines()
print(len(lines))
print(lines[0].split('\t')[0])

100
이


In [21]:
# 방법 1
# stopwords = list(map(lambda s: s.split('\t')[0], lines))
# 방법 2
stop_words = [line.split('\t')[0] for line in lines]

- 형태소 분석기를 돌려서 나온 결과에서 stop_words 제거하기

In [22]:
# test text 예시
morphs = okt.morphs(text, stem=True)
clean_morphs = [word for word in morphs if word not in stop_words]
clean_morphs

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [23]:
# clean_morphs를 문자열로 만들어 주어야 CountVectorizer의 입력으로 사용할 수 있음
clean_review = ' '.join(clean_morphs)
clean_review

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

In [24]:
# 한번에
review = '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'
morphs = okt.morphs(review, stem=True)
clean_morphs = [word for word in morphs if word not in stop_words]
clean_review = ' '.join(clean_morphs)
clean_review

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

- train/test dataset 적용

In [25]:
from tqdm import tqdm
X_train = []
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True)
    clean_morphs = [word for word in morphs if word not in stop_words]
    clean_morphs_review = ' '.join(clean_morphs)
    X_train.append(clean_morphs_review)

  0%|          | 0/145393 [00:00<?, ?it/s]

100%|██████████| 145393/145393 [05:51<00:00, 413.93it/s]


In [26]:
%%time
X_test = []
for review in test_df.document:
    morphs = okt.morphs(review, stem=True)
    clean_morphs = [word for word in morphs if word not in stop_words]
    clean_morphs_review = ' '.join(clean_morphs)
    X_test.append(clean_morphs_review)

CPU times: total: 1min 54s
Wall time: 1min 54s


In [27]:
X_train[:10]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정',
 '사이 몬페 의 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 던스트 너무나도 이쁘다 보이다',
 '막 걸음 마 떼다 세 부터 초등학교 학년 생인 살다 영화 ㅋㅋㅋ 별 반개 도 아깝다 움',
 '원작 의 긴장감 을 제대로 살리다 하다',
 '별 반개 도 아깝다 욕 나오다 이응경 길용우 연 기 생활 몇 인지 정말 발 로 해도 보단 낫다 납치 감금 만 반복 반복 드라마 는 가족 도 없다 연기 못 하다 만 모 엿 네',
 '액션 없다 재미 있다 몇 안되다 영화',
 '왜케 평점 낮다 꽤 볼 만 헐리우드 식 화려하다 너무 길들이다 있다']

In [28]:
y_train = train_df.label.values
y_test = test_df.label.values

##### 4. 피쳐 변환 + 모델 학습/평가

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [31]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('nb', MultinomialNB())
])

%time pipeline.fit(X_train, y_train)

CPU times: total: 2.16 s
Wall time: 2.03 s


Pipeline(steps=[('cvect', CountVectorizer()), ('nb', MultinomialNB())])

In [32]:
pipeline.score(X_test, y_test)

0.827581265864243

##### 5. 실제 데이터 테스트

In [33]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '이 영화 핵노잼 ㅠㅠ', 
    '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?',
    '와 개쩐다 정말 세계관 최강자들의 영화다'
]


In [34]:
# 전처리 - 한글 이외의 문자는 공백으로 변환
import re
reviews = list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', s), reviews))
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다']

In [35]:
# 형태소 분석 후 불용어 제거
processed_reviews = []
for review in reviews:
    morphs = okt.morphs(review, stem=True)
    clean_morphs = [word for word in morphs if word not in stop_words]
    clean_morphs_review = ' '.join(clean_morphs)
    processed_reviews.append(clean_morphs_review)

processed_reviews

['영화 개꿀잼 ㅋㅋㅋ',
 '영화 핵 노잼 ㅠㅠ',
 '이딴 게 영화 냐 ㅉㅉ',
 '감독 뭐 하다 놈 이냐',
 '오다 쩐다 정말 세계관 최강 자 의 영화 다']

In [36]:
pipeline.predict(processed_reviews)

array([1, 0, 0, 0, 1], dtype=int64)

In [37]:
pipeline.predict_proba(processed_reviews)

array([[0.06260368, 0.93739632],
       [0.91612638, 0.08387362],
       [0.9906787 , 0.0093213 ],
       [0.9297786 , 0.0702214 ],
       [0.09745728, 0.90254272]])

##### 6. 최적 파라메터 찾기

In [47]:
from sklearn.model_selection import GridSearchCV
params = {
    'cvect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'cvect__max_df': [0.91, 0.92, 0.93, 0.94]
}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=5)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 5min 37s
Wall time: 5min 24s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvect', CountVectorizer()),
                                       ('nb', MultinomialNB())]),
             param_grid={'cvect__max_df': [0.91, 0.92, 0.93, 0.94],
                         'cvect__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='accuracy')

In [48]:
grid_pipe.best_params_

{'cvect__max_df': 0.91, 'cvect__ngram_range': (1, 3)}

In [49]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8457586178662081

In [50]:
# 실제 데이터 적용
grid_pipe.best_estimator_.predict(processed_reviews)

array([1, 0, 0, 0, 1], dtype=int64)

- 다음 작업을 위한 준비

In [51]:
df = pd.DataFrame( {'x': X_train, 'y' : y_train})
df.head()

,x,y
0,아 더빙 진짜 짜증나다 목소리,0
1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다,1
2,너 무재 밓었 다그 래서 보다 추천 다,0
3,교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정,0
4,사이 몬페 의 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스...,1


In [52]:
df.to_csv('../data/네이버영화리뷰_train_형태소분석완료.csv', index=False)

In [53]:
df = pd.DataFrame( {'x': X_test, 'y' : y_test})
df.to_csv('../data/네이버영화리뷰_test_형태소분석완료.csv', index=False)